Import required libraries and modules


In [23]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense

Read in the data, saved temporarily in content folder. Needs to be imported for each runtime.

In [4]:
df = pd.read_csv('../content/Brain_GSE50161.csv', error_bad_lines=True)
df.shape

(130, 54677)

Select original X features and y labels. One-hot encode labels as numerical data. Split our features and labels into train and test splits. Initiate scaler on train data and transform train and test.

In [53]:
features = df.drop(['samples', 'type'], axis=1).columns

X = df.loc[:, features].values
y = pd.get_dummies(df.type)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1)

scaler = StandardScaler()
scaler = scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

Principal Component Analysis of the train and test data. We tried various numbers for n and found 20 to be the lowest reliable number.

In [54]:
pca = PCA(n_components=20)

pca = pca.fit(X_train_scaled)

X_train_scaled_pca = pca.transform(X_train_scaled)
X_test_scaled_pca = pca.transform(X_test_scaled)

Initiate the Keras Sequential model with three Dense layers.

In [55]:
ncols = X_train_scaled_pca.shape[1]

model = Sequential()
model.add(Dense(120, input_shape=(ncols,), activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Fit our model and print out accuracy.

In [57]:
model.fit(X_train_scaled_pca, y_train, epochs=200, batch_size=10, verbose=0)
scores = model.evaluate(X_test_scaled_pca, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

1/1 [==============================] - 0s 16ms/step - loss: 0.0068 - accuracy: 1.0000

accuracy: 100.00%
